# RAGCeption

This code is adapted from: https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_5_to_9.ipynb

In [1]:
from dotenv import load_dotenv
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.load import dumps, loads
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from operator import itemgetter

In [4]:
load_dotenv(override=True)

True

## Common Functions

### Loader, Splitter, Vectorstore, Retriever

In [31]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)


vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

### Declare LLM

In [ ]:
llm = ChatOpenAI(temperature=0)

### Prompt to Paraphrase Query

In [54]:
# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
paraphrase_prompt = ChatPromptTemplate.from_template(template)

### Prompt to Generate Result

In [53]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""
result_prompt = ChatPromptTemplate.from_template(template)

## Multi Query

In [55]:
# To get unique union of retrieved docs
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]


## RAG-Fusion

In [56]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results


## Combined

In [51]:
def function_selector(function_type: str, question: str):
    """ Function selector to select the appropriate function based on the question """
    generate_queries = (
        paraphrase_prompt 
        | llm 
        | StrOutputParser() 
        | (lambda x: x.split("\n"))
    )

    if function_type == "ranked":
        retrieval_chain = generate_queries | retriever.map() | reciprocal_rank_fusion
    elif function_type == "unique":
        retrieval_chain = generate_queries | retriever.map() | get_unique_union

    final_rag_chain = (
        {"context": retrieval_chain, 
        "question": itemgetter("question")} 
        | result_prompt
        | llm
        | StrOutputParser()
    )

    return final_rag_chain.invoke({"question":question})


In [52]:
# Retrieve
function_type = "ranked" # or "unique"
question = "What is task decomposition for LLM agents?"

function_selector(function_type="ranked", question=question)


'Task decomposition for LLM agents involves breaking down complex tasks into smaller and simpler steps to enhance model performance. This can be achieved through techniques such as Chain of Thought (CoT) and Tree of Thoughts, which prompt the model to think step by step and explore multiple reasoning possibilities at each step. Task decomposition can be done using simple prompting, task-specific instructions, or with human inputs.'